In [ ]:
!python -m venv sklearn-env # create virtual environment
!sklearn-env\Scripts\activate  # activate the environment
!pip install -U scikit-learn # install sklearn

  Using cached scikit_learn-1.7.2-cp311-cp311-win_amd64.whl (8.9 MB)
  Using cached numpy-2.3.4-cp311-cp311-win_amd64.whl (13.1 MB)
  Using cached scipy-1.16.3-cp311-cp311-win_amd64.whl (38.7 MB)
  Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()